# Image reconstruction using CG-SENSE

[![View on website](https://img.shields.io/badge/-View%20on%20website-128091?labelColor=grey&logo=)](https://github.com/mrphys/tensorflow-mri)
[![Run in Colab](https://img.shields.io/badge/-Run%20in%20Colab-128091?labelColor=grey&logo=googlecolab)](https://colab.research.google.com/github/mrphys/tensorflow-mri/blob/develop/tools/docs/tutorials/recon/cg_sense.ipynb)
[![View on GitHub](https://img.shields.io/badge/-View%20on%20GitHub-128091?labelColor=grey&logo=github)](https://github.com/mrphys/tensorflow-mri/blob/develop/tools/docs/tutorials/recon/cg_sense.ipynb)
[![Download notebook](https://img.shields.io/badge/-Download%20notebook-128091?labelColor=grey&logo=)](https://raw.githubusercontent.com/mrphys/tensorflow-mri/develop/tools/docs/tutorials/recon/cg_sense.ipynb)

## Set up TensorFlow MRI
If you have not yet installed TensorFlow MRI in your environment, you may do so
now using `pip`: 

In [1]:
%pip install tensorflow-mri

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Then, import the package into your program to get started:

In [2]:
import tensorflow_mri as tfmri
print("TensorFlow MRI version:", tfmri.__version__)

TensorFlow MRI version: 0.16.0


We will also need a few additional packages:

In [3]:
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

## Prepare the data

In [4]:
brain_data_filename = 'rawdata_brain_radial_96proj_12ch.h5'
brain_data_url = "https://github.com/ISMRM/rrsg/raw/master/challenges/challenge_01/rawdata_brain_radial_96proj_12ch.h5"
!wget -O {brain_data_filename} {brain_data_url}

--2022-04-20 15:16:07--  https://github.com/ISMRM/rrsg/raw/master/challenges/challenge_01/rawdata_brain_radial_96proj_12ch.h5
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ISMRM/rrsg/master/challenges/challenge_01/rawdata_brain_radial_96proj_12ch.h5 [following]
--2022-04-20 15:16:08--  https://raw.githubusercontent.com/ISMRM/rrsg/master/challenges/challenge_01/rawdata_brain_radial_96proj_12ch.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5310560 (5.1M) [application/octet-stream]
Saving to: ‘rawdata_brain_radial_96proj_12ch.h5’

rawdata_brain_radia 100%[===================>]   5.06M  5.80M

In [5]:
with h5py.File('rawdata_brain_radial_96proj_12ch.h5', 'r') as f:
  kspace = f['rawdata'][()]
  trajectory = f['trajectory'][()]

image_shape = [256, 256]

In [6]:
print(kspace.shape, trajectory.shape)

(1, 512, 96, 12) (3, 512, 96)


In [7]:
kspace = tf.squeeze(kspace, axis=0)
kspace = tf.transpose(kspace)
kspace = tf.reshape(kspace, [12, -1])

2022-04-20 15:16:09.491578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 15:16:09.497787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 15:16:09.497917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 15:16:09.498840: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
trajectory = trajectory[:2, ...]
trajectory = tf.transpose(trajectory)
trajectory = tf.reshape(trajectory, [-1, 2])

In [9]:
print(kspace.shape, trajectory.shape)

(12, 49152) (49152, 2)


In [ ]:
# Zero-filled reconstruction.
image = tfmri.recon.adj(kspace, image_shape, trajectory=trajectory)

## Compute coil sensitivity maps

In [10]:
# filter_fn = lambda x: tfmri.signal.hann(6 * x)

# low_res_kspace = tfmri.signal.filter_kspace(kspace,
#                                             traj=trajectory,
#                                             filter_fn=filter_fn)

# low_res_images = tfmri.recon.adj(low_res_kspace,
#                                  image_shape,
#                                  trajectory=trajectory)

# sensitivities = tfmri.coils.estimate_sensitivities()